In [18]:
import random
import math

# Data destinasi wisata
destinations = [
    {'no': 1, 'nama': 'destinasi 1', 'harga': 250, 'rating': 3.2},
    {'no': 2, 'nama': 'destinasi 2', 'harga': 170, 'rating': 4.6},
    {'no': 3, 'nama': 'destinasi 3', 'harga': 150, 'rating': 3.7},
    {'no': 4, 'nama': 'destinasi 4', 'harga': 420, 'rating': 5.0},
    {'no': 5, 'nama': 'destinasi 5', 'harga': 40, 'rating': 2.7},
    {'no': 6, 'nama': 'destinasi 6', 'harga': 300, 'rating': 4.9},
    {'no': 7, 'nama': 'destinasi 7', 'harga': 230, 'rating': 4.5},
    {'no': 8, 'nama': 'destinasi 8', 'harga': 400, 'rating': 1.0},
    {'no': 9, 'nama': 'destinasi 9', 'harga': 190, 'rating': 4.1},
    {'no': 10, 'nama': 'destinasi 10', 'harga': 200, 'rating': 3.4}
]

# Batas anggaran
max_cost = 750

In [19]:
# func utk hitung fitness (total rating) dari chromosome
def calculate_fitness(chromosome):
    total_cost = 0
    total_rating = 0
    
    for i in range(len(chromosome)):
        if chromosome[i] == 1:  # jika destinasi dipilih
            total_cost += destinations[i]['harga']
            total_rating += destinations[i]['rating']
    
    # Jika melebihi budget, berikan nilai fitness 0
    if total_cost > max_cost:
        return 0
    else:
        return total_rating


In [20]:
# func utk random population
def generate_population(size, length):
    population = []
    
    for i in range(size):
        genes = [0, 1]  # 0 = tidak dipilih, 1 = dipilih
        chromosome = []
        
        for _ in range(length):
            chromosome.append(random.choice(genes))
        
        individu = {
            'chromosome': chromosome,
            'fitness': calculate_fitness(chromosome)
        }
        
        population.append(individu)
    
    print(f"Generated a random population of size {size}")
    return population


In [21]:
# func untuk mengurutkan populasi berdasarkan fitness (dari terbesar ke terkecil)
from operator import itemgetter

def sort_population(population):
    return sorted(population, key=itemgetter('fitness'), reverse=True)

In [22]:
# func utk pilih chromosomes yg di crossover
def select_chromosomes(population, crossover_probability):
    fitness_values = []
    for i in range(len(population)):
        fitness_values.append(population[i]['fitness'])
    
    # Jika semua fitness 0, gunakan probabilitas yang sama
    if sum(fitness_values) == 0:
        probability = [1/len(fitness_values) for _ in fitness_values]
    else:
        probability = [float(fitness)/sum(fitness_values) for fitness in fitness_values]
    
    jumlah_kawin = math.ceil(crossover_probability * len(population))
    
    parents = random.choices(population, weights=probability, k=jumlah_kawin)
    
    return parents

In [23]:
# func utk crossover 2 kromosom
def crossover(chromosome1, chromosome2):
    chromosome_length = len(chromosome1)
    
    # Pastikan panjang kromosom cukup untuk crossover
    if chromosome_length < 4:
        crossover_point = 1
    else:
        crossover_point = random.randint(1, chromosome_length - 2)
    
    child1_chromosome = chromosome1[0:crossover_point] + chromosome2[crossover_point:]
    child2_chromosome = chromosome2[0:crossover_point] + chromosome1[crossover_point:]
    
    child1 = {
        'chromosome': child1_chromosome,
        'fitness': calculate_fitness(child1_chromosome)
    }
    
    child2 = {
        'chromosome': child2_chromosome,
        'fitness': calculate_fitness(child2_chromosome)
    }
    
    return child1, child2

In [24]:
# func utk mutasi chromosome
def mutate(chromosome, mutation_probability):
    for i in range(len(chromosome)):
        if random.uniform(0, 1) < mutation_probability:
            chromosome[i] = 1 - chromosome[i]  # Flip bit (0->1 atau 1->0)
    
    return chromosome

In [25]:
# func BGA
def eliminate(population, top_population, count):
    index = set()
    indexize = 0
    
    while indexize < count:
        bil = random.randint(0, len(population) - 1)
        if bil not in index:
            indexize += 1
            index.add(bil)
    
    j = 0
    for i in index:
        population[i] = top_population[j]
        j += 1
    
    return population

In [26]:
# func utk tampilkan hasil
def display_selected_destinations(best_chromosome):
    print("\n====== HASIL OPTIMASI ALGORITMA GENETIKA ======")
    print("Destinasi yang dikunjungi:")
    print("\nNo.  Nama            Harga (Rp)    Rating")
    print("-" * 50)
    
    total_cost = 0
    total_rating = 0
    count = 1
    
    for i in range(len(best_chromosome)):
        if best_chromosome[i] == 1:
            dest = destinations[i]
            print(f"{count:<4} {dest['nama']:<15} {dest['harga']:<13} {dest['rating']}")
            total_cost += dest['harga']
            total_rating += dest['rating']
            count += 1
    
    print("-" * 50)
    print(f"Total Harga: Rp {total_cost}")
    print(f"Total Rating (Fitness): {total_rating}")
    print(f"Anggaran Maksimum: Rp {max_cost}")
    print("=" * 50)


In [27]:
# main program
# Parameters for genetic algorithm
population_size = 100
crossover_probability = 0.8
mutation_probability = 0.1
generations = 100
r_bga = 0.1
jumlah_lestari = int(r_bga * population_size)

print("Daftar Destinasi Wisata:")
print("\nNo.  Nama            Harga (Rp)    Rating")
print("-" * 50)
for dest in destinations:
    print(f"{dest['no']:<4} {dest['nama']:<15} {dest['harga']:<13} {dest['rating']}")
print(f"Anggaran Maksimum: Rp {max_cost}")

print("\nGenetic Algorithm Parameters:")
print(f"Population: {population_size}")
print(f"Mutation probability: {mutation_probability}")
print(f"Crossover probability: {crossover_probability}")
print(f"Generations: {generations}")
print(f"Value of r for BGA: {r_bga} sehingga jumlah individu yang dilestarikan {jumlah_lestari}")

print("\nPerforming genetic evolution:")

# Generate a random population
population = generate_population(population_size, len(destinations))
# Sort population by fitness
population = sort_population(population)
# Get top population
top_population = population[0:(jumlah_lestari + 1)]

population

Daftar Destinasi Wisata:

No.  Nama            Harga (Rp)    Rating
--------------------------------------------------
1    destinasi 1     250           3.2
2    destinasi 2     170           4.6
3    destinasi 3     150           3.7
4    destinasi 4     420           5.0
5    destinasi 5     40            2.7
6    destinasi 6     300           4.9
7    destinasi 7     230           4.5
8    destinasi 8     400           1.0
9    destinasi 9     190           4.1
10   destinasi 10    200           3.4
Anggaran Maksimum: Rp 750

Genetic Algorithm Parameters:
Population: 100
Mutation probability: 0.1
Crossover probability: 0.8
Generations: 100
Value of r for BGA: 0.1 sehingga jumlah individu yang dilestarikan 10

Performing genetic evolution:
Generated a random population of size 100


[{'chromosome': [0, 0, 0, 0, 1, 0, 1, 0, 1, 1], 'fitness': 14.700000000000001},
 {'chromosome': [0, 1, 1, 0, 0, 1, 0, 0, 0, 0], 'fitness': 13.200000000000001},
 {'chromosome': [0, 0, 0, 0, 1, 1, 0, 0, 1, 0], 'fitness': 11.7},
 {'chromosome': [0, 0, 1, 0, 1, 1, 0, 0, 0, 0], 'fitness': 11.3},
 {'chromosome': [1, 1, 0, 0, 0, 0, 0, 0, 0, 1], 'fitness': 11.2},
 {'chromosome': [1, 0, 0, 0, 1, 1, 0, 0, 0, 0], 'fitness': 10.8},
 {'chromosome': [0, 0, 0, 1, 0, 1, 0, 0, 0, 0], 'fitness': 9.9},
 {'chromosome': [1, 0, 0, 0, 1, 0, 0, 0, 0, 1], 'fitness': 9.3},
 {'chromosome': [0, 0, 0, 0, 0, 0, 1, 0, 1, 0], 'fitness': 8.6},
 {'chromosome': [1, 0, 0, 0, 0, 0, 0, 0, 1, 0], 'fitness': 7.3},
 {'chromosome': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0], 'fitness': 5.0},
 {'chromosome': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'fitness': 3.2},
 {'chromosome': [0, 0, 1, 0, 1, 0, 1, 0, 1, 1], 'fitness': 0},
 {'chromosome': [1, 0, 0, 1, 0, 0, 1, 0, 1, 1], 'fitness': 0},
 {'chromosome': [1, 1, 1, 1, 0, 1, 1, 1, 1, 1], 'fitness': 0

In [28]:
# Evolution process
for i in range(generations):
    print(f"\nGeneration: {i+1}")
    
    # Select parents
    parents = select_chromosomes(population, crossover_probability)
    
    # Remove parents from population
    for par in range(len(parents)):
        for popu in range(len(population)):
            if parents[par] == population[popu]:
                population.pop(popu)
                break
    
    # Crossover
    for cross in range(len(parents) // 2):
        if cross * 2 + 1 < len(parents):  # Make sure we have pairs
            child1, child2 = crossover(parents[0 + 2 * cross]['chromosome'], parents[1 + 2 * cross]['chromosome'])
            
            # Mutation
            child1['chromosome'] = mutate(child1['chromosome'], mutation_probability)
            child2['chromosome'] = mutate(child2['chromosome'], mutation_probability)
            
            # Recalculate fitness after mutation
            child1['fitness'] = calculate_fitness(child1['chromosome'])
            child2['fitness'] = calculate_fitness(child2['chromosome'])
            
            # Add children to population
            population.append(child1)
            population.append(child2)
    
    # BGA - Elitism
    population = eliminate(population, top_population, jumlah_lestari)
    
    # Sort population by fitness
    population = sort_population(population)
    
    # Ensure population size remains constant
    if len(population) > population_size:
        population = population[0:population_size]
    
    # Update top population
    top_population = population[0:(jumlah_lestari + 1)]
    
    # Print best individual in this generation
    print(f"Best Fitness: {population[0]['fitness']}")

print("\nGenetic evolution complete!")
best_solution = population[0]
print(f"Best solution fitness: {best_solution['fitness']}")

# Display the selected destinations
display_selected_destinations(best_solution['chromosome'])


Generation: 1
Best Fitness: 15.4

Generation: 2
Best Fitness: 16.7

Generation: 3
Best Fitness: 16.7

Generation: 4
Best Fitness: 16.7

Generation: 5
Best Fitness: 16.7

Generation: 6
Best Fitness: 16.7

Generation: 7
Best Fitness: 16.7

Generation: 8
Best Fitness: 16.7

Generation: 9
Best Fitness: 16.7

Generation: 10
Best Fitness: 16.7

Generation: 11
Best Fitness: 18.5

Generation: 12
Best Fitness: 18.5

Generation: 13
Best Fitness: 18.5

Generation: 14
Best Fitness: 18.5

Generation: 15
Best Fitness: 18.5

Generation: 16
Best Fitness: 18.5

Generation: 17
Best Fitness: 18.5

Generation: 18
Best Fitness: 18.5

Generation: 19
Best Fitness: 18.5

Generation: 20
Best Fitness: 18.5

Generation: 21
Best Fitness: 18.5

Generation: 22
Best Fitness: 18.5

Generation: 23
Best Fitness: 18.5

Generation: 24
Best Fitness: 18.5

Generation: 25
Best Fitness: 18.5

Generation: 26
Best Fitness: 18.5

Generation: 27
Best Fitness: 18.5

Generation: 28
Best Fitness: 18.5

Generation: 29
Best Fitness: